In [29]:
!pip install vaderSentiment

In [2]:
import csv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os.path

data_sufix = 'data/'

In [3]:
# configure Session class with desired options
Session = sessionmaker()
engine = create_engine('sqlite:///twitter.db')

In [4]:
engine.table_names()

['hashtag', 'retweet', 'tweet', 'user', 'vader']

In [4]:
hashtags = ['h1b', 'usvisa', 'immigrationreform', 'illegalImmigration', 'immigration',
'immigrantcaravan', 'caravaninvasion', 'buildthewall']
remaining_hashtags = ['illegalaliens', 'birthrightcitizenship']
hashtagsDict = {'name': hashtags}

In [5]:
hashtagData = pd.DataFrame.from_dict(hashtagsDict)
hashtagData.to_sql('hashtag', engine, if_exists='replace', index=True, index_label='id')

In [11]:
for hashtag in hashtags:
    
    hashtag_id = pd.read_sql_query('SELECT id FROM hashtag WHERE hashtag.name="'+hashtag+'"', engine).values[0][0]

    tweetData = pd.read_csv(data_sufix+hashtag+'-tweet.csv', 
                            usecols=['tweet_id','tweet_text','tweet_created_at','tweet_retweet_count',
                                     'tweet_favorite_count','user_id'])
    tweetData['hashtag_id'] = hashtag_id
    
    retweetData = pd.read_csv(data_sufix+hashtag+'-retweet.csv', 
                              usecols=['retweet_id','source_tweet_id','retweet_text','retweet_created_at','user_id'])
    
    tweetData.to_sql('tweet', engine, if_exists='append')
    retweetData.to_sql('retweet', engine, if_exists='append')

In [12]:
userData = pd.read_csv(data_sufix+'user.csv', 
                        usecols=['user_id','user_name','user_location','user_friends_count','followers_count'])

userData.to_sql('user', engine, if_exists='append')

In [5]:
#VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

allTweets = pd.read_sql_query('SELECT tweet_id as id, tweet_text as text FROM tweet', engine)
print(allTweets.shape)

(17220, 2)


In [6]:
allRetweets =  pd.read_sql_query('SELECT retweet_id as id, retweet_text as text FROM retweet', engine)
print(allRetweets.shape)

(77828, 2)


In [7]:
vaderData = allTweets.append([allRetweets])
print(vaderData.shape)


(95048, 2)


In [8]:
vaderIDs = pd.read_sql_query('SELECT id FROM vader', engine)
vaderIDs

,id
0,1056095192930025472
1,1056125697482256384
2,1056125759851556864
3,1056125800196595712
4,1056125844966592512
5,1056126058175647744
6,1056138781626118144
7,1056152900727119873
8,1056153754863652864
9,1056165457491906561


In [10]:
newVaders = pd.DataFrame(columns=['id', 'negative','neutral','positive','compound'])

In [11]:
for index, row in vaderData.iterrows():
    if(not vaderIDs['id'].isin([row['id']]).any()):
        snt = analyser.polarity_scores(row['text'])
        newVaders = newVaders.append({'id': row['id'],
                                     'negative': snt['neg'],
                                     'neutral': snt['neu'],
                                     'positive': snt['pos'],
                                     'compound': snt['compound']}, ignore_index=True)
newVaders

,id,negative,neutral,positive,compound
0,1.056175e+18,0.000,0.863,0.137,0.4019
1,1.056199e+18,0.000,0.832,0.168,0.4995
2,1.056206e+18,0.000,1.000,0.000,0.0000
3,1.056210e+18,0.000,0.856,0.144,0.4019
4,1.056210e+18,0.179,0.821,0.000,-0.5719
5,1.056212e+18,0.171,0.829,0.000,-0.4767
6,1.056216e+18,0.000,0.892,0.108,0.4404
7,1.056217e+18,0.259,0.741,0.000,-0.6369
8,1.056221e+18,0.117,0.778,0.105,-0.0772
9,1.056222e+18,0.000,0.872,0.128,0.4215


In [48]:
newVaders.to_sql('vader', engine, if_exists='append')

In [5]:
# associate it with our custom Session class
Session.configure(bind=engine)